In [14]:
!pip install ultralytics

In [15]:
!pip install numpy scipy scikit-learn matplotlib pandas tqdm opencv-python

In [16]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [17]:
!pip install torch-geometric

In [18]:
import torch
import torch.nn.functional as F
import numpy as np
import os
import shutil
from sklearn.metrics import pairwise_distances
from ultralytics import YOLO
from torch_geometric.nn import GATConv, GCNConv
from torch_geometric.data import Data
from torchvision import transforms
from sklearn.neighbors import kneighbors_graph
from torch_geometric.utils import add_self_loops, remove_self_loops
from PIL import Image
import torch.nn as nn

In [19]:
model = YOLO("yolo11s.pt")

model.train(
    data="/kaggle/input/yaml-files/dataset.yaml",
    epochs=500,
    imgsz=512,
    batch=16,
    lr0=0.0002,
    lrf=0.002,
    momentum=0.98,
    weight_decay=0.0001,
    optimizer="RAdam",
    cos_lr = True,
    warmup_epochs=30,
    iou=0.5,
    mosaic=0.0, 
    dfl = 2.0,
    device="cuda",
    augment=True,
    verbose=True
)

100%|██████████| 18.4M/18.4M [00:00<00:00, 122MB/s] 


Ultralytics 8.3.70 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/kaggle/input/yaml-files/dataset.yaml, epochs=500, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=RAdam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

train: Scanning /kaggle/input/dataset-weed/labeled/labels... 200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<00:00, 760.84it/s]

train: WARNING ⚠️ Cache directory /kaggle/input/dataset-weed/labeled is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/input/dataset-weed/test/labels... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<00:00, 429.85it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.23it/s]


                   all         50         74      0.906      0.875      0.896      0.592
                  weed         24         42      0.964      0.881      0.942      0.664
                  crop         26         32      0.848       0.87      0.851       0.52


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 14.6ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train6


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f63cd741e10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [36]:
def extract_features(image_paths,device="cuda"):
    features = []
    transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    for img_path in image_paths:
        img = transform(Image.open(img_path).convert("RGB")).unsqueeze(0).to(device)
        with torch.no_grad():
            feat1 = model.model.model[0:5](img)
            feat2 = model.model.model[0:10](img)
            
            feat1_pooled = torch.cat([
                torch.mean(feat1, dim=[2, 3]),
                torch.max(feat1, dim=3)[0].max(dim=2)[0]
            ], dim=1)
            
            feat2_pooled = torch.cat([
                torch.mean(feat2, dim=[2, 3]),
                torch.max(feat2, dim=3)[0].max(dim=2)[0]
            ], dim=1)
            
            combined_feat = torch.cat([feat1_pooled, feat2_pooled], dim=1)
            features.append(combined_feat.view(-1).cpu().numpy())
    
    features = np.array(features)
    features = (features - features.mean(axis=0)) / (features.std(axis=0) + 1e-6)
    return features

labeled_image_paths = ["/kaggle/input/dataset-weed/labeled/images/" + f for f in os.listdir("/kaggle/input/dataset-weed/labeled/images")]
unlabeled_image_paths = ["/kaggle/input/dataset-weed/unlabeled/" + f for f in os.listdir("/kaggle/input/dataset-weed/unlabeled")]

labeled_features = extract_features(labeled_image_paths)
unlabeled_features = extract_features(unlabeled_image_paths)

In [37]:
def construct_knn_graph(features, k=15):
    distances = pairwise_distances(features)
    sigma = np.mean(np.sort(distances, axis=1)[:, 1:k+1])
    weights = np.exp(-distances**2 / (2 * sigma**2))
    
    adjacency_matrix = kneighbors_graph(features, k, mode="connectivity", include_self=False)
    edge_index_np = np.array(adjacency_matrix.nonzero())
    edge_weights_np = weights[adjacency_matrix.nonzero()]
    edge_index = torch.tensor(edge_index_np, dtype=torch.long)
    edge_weights = torch.tensor(edge_weights_np, dtype=torch.float32)
    
    edge_index, edge_weights = add_self_loops(edge_index, edge_weights, num_nodes=len(features))
    return edge_index, edge_weights

edge_index, edge_weights = construct_knn_graph(np.vstack([labeled_features, unlabeled_features]))
num_nodes = len(labeled_features) + len(unlabeled_features)


In [38]:
def load_labels(label_paths):
    labels = []
    for label_path in label_paths:
        with open(label_path, "r") as f:
            class_ids = [int(line.split()[0]) for line in f.readlines()]
        labels.append(max(set(class_ids)) if class_ids else 0)
    return np.array(labels)

labeled_label_paths = [f"/kaggle/input/dataset-weed/labeled/labels/{os.path.basename(p).rsplit('.', 1)[0]}.txt" for p in labeled_image_paths]
actual_labels = load_labels(labeled_label_paths)
graph_data = Data(
    x=torch.tensor(np.vstack([labeled_features, unlabeled_features]), dtype=torch.float32),
    edge_index=edge_index,
    y=torch.cat([torch.tensor(actual_labels, dtype=torch.long), 
                torch.full((len(unlabeled_features),), -1)])
)

In [39]:
unique_labels, counts = np.unique(graph_data.y.numpy(), return_counts=True)
print("✅ Fixed Graph Data Labels:", dict(zip(unique_labels, counts)))

✅ Fixed Graph Data Labels: {-1: 1000, 0: 88, 1: 112}


In [40]:
class GNN(nn.Module):
    def __init__(self, in_dim, hidden_dim=128, num_classes=2):
        super(GNN, self).__init__()
        self.conv1 = GATConv(in_dim, hidden_dim, heads=8, dropout=0.5)
        self.conv2 = GATConv(hidden_dim * 8, hidden_dim, heads=8, dropout=0.5)
        self.conv3 = GCNConv(hidden_dim * 8, hidden_dim)
        self.lin = nn.Linear(hidden_dim, num_classes)
        self.batch_norm1 = nn.BatchNorm1d(hidden_dim * 8)
        self.batch_norm2 = nn.BatchNorm1d(hidden_dim * 8)
        self.batch_norm3 = nn.BatchNorm1d(hidden_dim)
        
    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv1(x, edge_index)
        x = self.batch_norm1(x)
        x = F.elu(x)
        
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = self.batch_norm2(x)
        x = F.elu(x)
        
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = self.batch_norm3(x)
        x = F.elu(x)
        
        x = self.lin(x)
        return F.log_softmax(x, dim=1)
gnn_model = GNN(in_dim=graph_data.x.shape[1])
optimizer = torch.optim.AdamW(gnn_model.parameters(), lr=0.0005, weight_decay=0.01, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=10, factor=0.5)


In [42]:
def calculate_loss(out, graph_data, pseudo_labels, pseudo_mask):
    labeled_mask = graph_data.y != -1
    supervised_loss = F.nll_loss(out[labeled_mask], graph_data.y[labeled_mask])
    
    pseudo_loss = 0.0
    if pseudo_mask.any():
        pseudo_loss = F.nll_loss(out[pseudo_mask], pseudo_labels[pseudo_mask])
    
    edge_index, _ = remove_self_loops(graph_data.edge_index)
    smoothness_loss = torch.mean(torch.pow(
        out[edge_index[0]] - out[edge_index[1]], 2
    ))
    
    pseudo_weight = min(0.3, gnn_model.training_step / 100)
    total_loss = supervised_loss + pseudo_weight * pseudo_loss + 0.1 * smoothness_loss
    
    return total_loss, {
        'supervised': supervised_loss.item(),
        'pseudo': pseudo_loss.item() if isinstance(pseudo_loss, torch.Tensor) else pseudo_loss,
        'smoothness': smoothness_loss.item(),
        'total': total_loss.item()
    }

In [43]:
def generate_pseudo_labels(model, graph_data, confidence_threshold=0.9):
    model.eval()
    with torch.no_grad():
        out = model(graph_data.x, graph_data.edge_index)
        probs = torch.exp(out)
        max_probs, pseudo_labels = probs.max(dim=1)
        
        unlabeled_mask = graph_data.y == -1
        mean_confidence = max_probs[unlabeled_mask].mean()
        adaptive_threshold = min(confidence_threshold, mean_confidence + 0.1)
        
        confidence_mask = (max_probs > adaptive_threshold) & unlabeled_mask
        
        if confidence_mask.any():
            class_dist = torch.bincount(pseudo_labels[confidence_mask])
            if len(class_dist) > 1:
                ratio = float(class_dist.max()) / class_dist.min()
                if ratio > 3:
                    minor_class = class_dist.argmin()
                    confidence_mask &= ((pseudo_labels == minor_class) | 
                                     (max_probs > (adaptive_threshold + 0.1)))
        
        return pseudo_labels, confidence_mask

In [45]:
gnn_model.training_step = 0
best_loss = float('inf')

for epoch in range(250):
    gnn_model.train()
    optimizer.zero_grad()
    out = gnn_model(graph_data.x, graph_data.edge_index)
    
    pseudo_labels, confidence_mask = generate_pseudo_labels(gnn_model, graph_data)
    loss, loss_components = calculate_loss(out, graph_data, pseudo_labels, confidence_mask)
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(gnn_model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step(loss_components['total'])
    
    gnn_model.training_step += 1
    
    if loss_components['total'] < best_loss:
        best_loss = loss_components['total']
        torch.save(gnn_model.state_dict(), '/kaggle/working/WEED.pt')
    
    if epoch % 10 == 0:
        print(f"\nEpoch {epoch}:")
        for k, v in loss_components.items():
            print(f"{k.capitalize()} Loss: {v:.4f}")
        print(f"Confident Pseudo Labels: {confidence_mask.sum().item()}")
        if confidence_mask.any():
            print(f"Class distribution: {torch.bincount(pseudo_labels[confidence_mask])}")

gnn_model.load_state_dict(torch.load('/kaggle/working/WEED.pt'))
gnn_model.eval()

with torch.no_grad():
    final_out = gnn_model(graph_data.x, graph_data.edge_index)
    pseudo_labels = final_out.argmax(dim=1)
    confidence_mask = torch.exp(final_out).max(dim=1)[0] > 0.9



Epoch 0:
Supervised Loss: 0.8005
Pseudo Loss: 0.4802
Smoothness Loss: 0.0715
Total Loss: 0.8077
Confident Pseudo Labels: 93
Class distribution: tensor([93])

Epoch 10:
Supervised Loss: 0.0931
Pseudo Loss: 0.0695
Smoothness Loss: 0.5378
Total Loss: 0.1538
Confident Pseudo Labels: 916
Class distribution: tensor([417, 499])

Epoch 20:
Supervised Loss: 0.0786
Pseudo Loss: 0.0578
Smoothness Loss: 0.4799
Total Loss: 0.1381
Confident Pseudo Labels: 896
Class distribution: tensor([409, 487])

Epoch 30:
Supervised Loss: 0.0639
Pseudo Loss: 0.0455
Smoothness Loss: 0.4730
Total Loss: 0.1249
Confident Pseudo Labels: 896
Class distribution: tensor([416, 480])

Epoch 40:
Supervised Loss: 0.0587
Pseudo Loss: 0.0407
Smoothness Loss: 0.4327
Total Loss: 0.1142
Confident Pseudo Labels: 907
Class distribution: tensor([428, 479])

Epoch 50:
Supervised Loss: 0.0558
Pseudo Loss: 0.0343
Smoothness Loss: 0.4635
Total Loss: 0.1125
Confident Pseudo Labels: 910
Class distribution: tensor([428, 482])

Epoch 60:
S

In [46]:
unique_preds, counts_preds = np.unique(pseudo_labels, return_counts=True)
print("✅ Predicted class distribution:", dict(zip(unique_preds, counts_preds)))

✅ Predicted class distribution: {0: 561, 1: 639}


In [47]:
print("graph_data.x shape:", graph_data.x.shape)
print("Max index in edge_index:", edge_index.max().item())

graph_data.x shape: torch.Size([1200, 1536])
Max index in edge_index: 1199


In [48]:
def save_pseudo_labels(pseudo_labels, confidence_mask, unlabeled_image_paths, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    for idx, (img_path, label, is_confident) in enumerate(zip(
            unlabeled_image_paths, 
            pseudo_labels[len(labeled_image_paths):], 
            confidence_mask[len(labeled_image_paths):])):
        
        if is_confident:
            img_name = os.path.basename(img_path).rsplit('.', 1)[0]
            label_path = os.path.join(output_dir, f"{img_name}.txt")
            results = model(img_path)
            
            with open(label_path, "w") as f:
                for result in results:
                    for box in result.boxes:
                        if box.conf > 0.6:
                            x_center, y_center, width, height = box.xywhn.cpu().numpy().flatten()
                            f.write(f"{int(label.item())} {x_center} {y_center} {width} {height}\n")

save_pseudo_labels(
    pseudo_labels,
    confidence_mask,
    unlabeled_image_paths,
    "/kaggle/working/unlabeled/labels/"
)


image 1/1 /kaggle/input/dataset-weed/unlabeled/agri_0_5225_jpeg.rf.ab0d818fca3b3b0ef3411dea1a56000f.jpg: 512x512 1 crop, 34.5ms
Speed: 1.6ms preprocess, 34.5ms inference, 1.4ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /kaggle/input/dataset-weed/unlabeled/agri_0_2390_jpeg.rf.b4af55b63fe3ad51e5ccaa7fae52a15a.jpg: 512x512 1 crop, 28.4ms
Speed: 1.0ms preprocess, 28.4ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /kaggle/input/dataset-weed/unlabeled/agri_0_2633_jpeg.rf.80320a4fd05d32f7fae265c2d09ef7c4.jpg: 512x512 22 weeds, 28.7ms
Speed: 0.9ms preprocess, 28.7ms inference, 1.2ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /kaggle/input/dataset-weed/unlabeled/agri_0_1509_jpeg.rf.0d24077902e5229abad25227f9b58df9.jpg: 512x512 2 crops, 28.4ms
Speed: 0.9ms preprocess, 28.4ms inference, 1.1ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /kaggle/input/dataset-weed/unlabeled/agri_0_5384_jpeg.rf.225f34308f10dabf4afb9fbd5

In [49]:
os.makedirs("/kaggle/working/dataset_updated/images", exist_ok=True)
os.makedirs("/kaggle/working/dataset_updated/labels", exist_ok=True)
os.makedirs("/kaggle/working/test", exist_ok=True)
for file in os.listdir("/kaggle/input/dataset-weed/labeled/images/"):
    src = os.path.join("/kaggle/input/dataset-weed/labeled/images", file)
    dst = os.path.join("/kaggle/working/dataset_updated/images", file)
    if os.path.isfile(src):
        shutil.copy(src, dst)

for file in os.listdir("/kaggle/input/dataset-weed/unlabeled/"):
    src = os.path.join("/kaggle/input/dataset-weed/unlabeled", file)
    dst = os.path.join("/kaggle/working/dataset_updated/images", file)
    if os.path.isfile(src):
        shutil.copy(src, dst)

shutil.copytree("/kaggle/working/unlabeled/labels/", "/kaggle/working/dataset_updated/labels/", dirs_exist_ok=True)
shutil.copytree("/kaggle/input/dataset-weed/labeled/labels/", "/kaggle/working/dataset_updated/labels/", dirs_exist_ok=True)
shutil.copytree("/kaggle/input/dataset-weed/test/", "/kaggle/working/test/", dirs_exist_ok=True)

'/kaggle/working/test/'

In [51]:
model.train(
    data="/kaggle/input/yaml-files/dataset_updated.yaml",
    epochs=500,
    imgsz=512,
    batch=64,
    lr0=0.0002,
    lrf=0.0025,
    momentum=0.98,
    weight_decay=0.0001,
    optimizer="RAdam",
    cos_lr = True,
    warmup_epochs=30,
    iou=0.5,
    mosaic=0.0,
    dfl = 1.5,
    augment=True,
    device="cuda"
)

Ultralytics 8.3.70 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/kaggle/input/yaml-files/dataset_updated.yaml, epochs=500, time=None, patience=100, batch=64, imgsz=512, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train64, exist_ok=False, pretrained=True, optimizer=RAdam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels

train: Scanning /kaggle/working/dataset_updated/labels.cache... 1162 images, 41 backgrounds, 0 corrupt: 100%|██████████| 1200/1200 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/test/labels.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train64/labels.jpg... 
optimizer: RAdam(lr=0.0002, momentum=0.98) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0001), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 512 train, 512 val
Using 4 dataloader workers
Logging results to runs/detect/train64
Starting training for 500 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.33it/s]


                   all         50         74       0.88       0.86      0.884      0.596
                  weed         24         42      0.913      0.857      0.928      0.646
                  crop         26         32      0.847      0.864       0.84      0.546


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 8.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train64


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f63a0fee4d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [58]:
model = YOLO("/kaggle/working/runs/detect/train64/weights/best.pt")
metrics = model.val(data="/kaggle/input/yaml-files/test.yaml", device="cuda")
precision = metrics.box.p.mean()  # ✅ Mean Precision
recall = metrics.box.r.mean()  # ✅ Mean Recall
map_50 = metrics.box.map50  # ✅ mAP@50
map_50_95 = metrics.box.map  # ✅ mAP@50-95

f1_score = (2 * precision * recall) / (precision + recall + 1e-6) 
print(f"✅ Mean Precision : {precision:.4f}")
print(f"✅ Mean Recall : {recall:.4f}")
print(f"✅ mAP@50 : {map_50:.4f}")
print(f"✅ mAP@50-95 : {map_50_95:.4f}")
print(f"✅ Mean F1-Score : {f1_score:.4f}")

final_score = 0.5*f1_score + 0.5*map_50_95
print(final_score)

Ultralytics 8.3.70 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11s summary (fused): 238 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /kaggle/input/dataset-weed/test/labels... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<00:00, 605.01it/s]

val: WARNING ⚠️ Cache directory /kaggle/input/dataset-weed/test is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.44it/s]


                   all         50         74      0.929      0.848      0.905      0.647
                  weed         24         42          1      0.852      0.901       0.68
                  crop         26         32      0.858      0.844      0.908      0.615


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 6.6ms inference, 0.0ms loss, 7.2ms postprocess per image
Results saved to runs/detect/val2
✅ Mean Precision : 0.9290
✅ Mean Recall : 0.8478
✅ mAP@50 : 0.9047
✅ mAP@50-95 : 0.6474
✅ Mean F1-Score : 0.8865
0.7669756699028349
